# [Hate Speech Identification Shared Task](https://multihate.github.io/): Subtask 1A at [BLP Workshop](https://blp-workshop.github.io/) @IJCNLP-AACL 2025

This shared task is designed to identify the type of hate, its severity, and the targeted group from social media content. The goal is to develop robust systems that advance research in this area.

In this subtask, given a Bangla text collected from YouTube comments, categorize whether it contains abusive, sexism, religious hate, political hate, profane, or none.

### Downloading dataset from github

In [1]:
!wget https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1A/blp25_hatespeech_subtask_1A_train.tsv
!wget https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1A/blp25_hatespeech_subtask_1A_dev.tsv
!wget https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1A/blp25_hatespeech_subtask_1A_dev_test.tsv

--2025-08-18 12:36:11--  https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1A/blp25_hatespeech_subtask_1A_train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8002036 (7.6M) [text/plain]
Saving to: ‘blp25_hatespeech_subtask_1A_train.tsv.2’

blp25_hatespeech_su 100%[===================>]   7.63M  --.-KB/s    in 0.08s   

2025-08-18 12:36:12 (94.8 MB/s) - ‘blp25_hatespeech_subtask_1A_train.tsv.2’ saved [8002036/8002036]

--2025-08-18 12:36:12--  https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1A/blp25_hatespeech_subtask_1A_dev.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubuserconten

### installing required libraries.
 - transformers
 - datasets
 - evaluate
 - accelerate

In [2]:
!pip install transformers
!pip install datasets
!pip install evaluate
# !pip install --upgrade accelerate

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


#### importing required libraries and setting up logger

In [1]:
import logging
import os
import random
import sys
from dataclasses import dataclass, field
from typing import Optional
import pandas as pd
import datasets
import evaluate
import numpy as np
from datasets import load_dataset, Dataset, DatasetDict
import torch

import transformers
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    PretrainedConfig,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version, send_example_telemetry
from transformers.utils.versions import require_version


logger = logging.getLogger(__name__)

logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)

### Defining the training, validation, and test data

In [2]:
train_file = 'blp25_hatespeech_subtask_1A_train.tsv'
validation_file = 'blp25_hatespeech_subtask_1A_dev.tsv'
test_file = 'blp25_hatespeech_subtask_1A_dev_test.tsv'

### Disable wandb

In [3]:
import os
os.environ["WANDB_DISABLED"] = "true"

### Setting up the training parameters

In [4]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./BERT_HateSpeech/",
    
    # Your core changes
    num_train_epochs=15,  # Increased from 1 to 3. You can experiment with 4 or 5.
    
    # Important adjustments for finding the best model
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    save_strategy="epoch",        # Save a checkpoint at the end of each epoch
    load_best_model_at_end=True,  # This will load the best checkpoint when training is done
    
    # Other common parameters
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    save_total_limit=2,           # Saves disk space by only keeping the best 2 checkpoints
    report_to="none",
    overwrite_output_dir=True,
)

max_train_samples = None
max_eval_samples=None
max_predict_samples=None
max_seq_length = 512
batch_size = 16

/users/t/p/tprama/.local/lib/python3.12/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [5]:
transformers.utils.logging.set_verbosity_info()

log_level = training_args.get_process_log_level()
logger.setLevel(log_level)
datasets.utils.logging.set_verbosity(log_level)
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()
logger.warning(
    f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
    + f" distributed training: {bool(training_args.local_rank != -1)}, 16-bits training: {training_args.fp16}"
)
logger.info(f"Training/evaluation parameters {training_args}")

08/20/2025 23:50:07 - WARNING - __main__ - Process rank: 0, device: cpu, n_gpu: 0 distributed training: True, 16-bits training: False
08/20/2025 23:50:07 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=0,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=Non

#### Defining the Model

In [33]:
model_name = 'aplycaebous/tb-BERT-fpt'

#model_name = "csebuetnlp/banglabert" 

#### setting the random seed

In [34]:
set_seed(training_args.seed)

#### Loading data files

In [35]:
l2id = {'None': 0, 'Religious Hate': 1, 'Sexism': 2, 'Political Hate': 3, 'Profane': 4, 'Abusive': 5}
train_df = pd.read_csv(train_file, sep='\t')
# print(train_df['label'])
train_df['label'] = train_df['label'].map(l2id).fillna(0).astype(int)
train_df = Dataset.from_pandas(train_df)
validation_df = pd.read_csv(validation_file, sep='\t')
validation_df['label'] = validation_df['label'].map(l2id).fillna(0).astype(int)
validation_df = Dataset.from_pandas(validation_df)
test_df = pd.read_csv(test_file, sep='\t')
#test_df['label'] = test_df['label'].map(l2id)
test_df = Dataset.from_pandas(test_df)

data_files = {"train": train_df, "validation": validation_df, "test": test_df}
for key in data_files.keys():
    logger.info(f"loading a local file for {key}")
raw_datasets = DatasetDict(
    {"train": train_df, "validation": validation_df, "test": test_df}
)

08/18/2025 12:50:50 - INFO - __main__ - loading a local file for train
08/18/2025 12:50:50 - INFO - __main__ - loading a local file for validation
08/18/2025 12:50:50 - INFO - __main__ - loading a local file for test


In [36]:
len(test_df['id'])

2512

##### Extracting number of unique labels

In [37]:
# Labels
label_list = raw_datasets["train"].unique("label")
print(label_list)
label_list.sort()  # sort the labels for determine
num_labels = len(label_list)

[0, 5, 4, 1, 3, 2]


### Loading Pretrained Configuration, Tokenizer and Model

In [38]:

from transformers import AutoConfig, AutoTokenizer, AutoModelForSequenceClassification

# Define your model name and number of labels
model_name = "sagorsarker/bangla-bert-base"
num_labels = 6 # Based on your l2id dictionary: None, Religious, Sexism, etc.

# Load the model configuration
config = AutoConfig.from_pretrained(
    model_name,
    num_labels=num_labels,
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
)

# Load the model FOR SEQUENCE CLASSIFICATION
# This is the key change!
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    config=config,
    ignore_mismatched_sizes=True, # Important when adding a new head
)

[INFO|configuration_utils.py:699] 2025-08-18 12:50:52,757 >> loading configuration file config.json from cache at /users/t/p/tprama/.cache/huggingface/hub/models--sagorsarker--bangla-bert-base/snapshots/875aa80a42ec196c16bd931ae5d85ad949f58b16/config.json
[INFO|configuration_utils.py:771] 2025-08-18 12:50:52,758 >> Model config BertConfig {
  "_name_or_path": "sagorsarker/bangla-bert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  

#### Preprocessing the raw_datasets

In [39]:
non_label_column_names = [name for name in raw_datasets["train"].column_names if name != "label"]
sentence1_key= non_label_column_names[1]

# Padding strategy
padding = "max_length"

# Some models have set the order of the labels to use, so let's make sure we do use it.
label_to_id = None
if (model.config.label2id != PretrainedConfig(num_labels=num_labels).label2id):
    # Some have all caps in their config, some don't.
    label_name_to_id = {k.lower(): v for k, v in model.config.label2id.items()}
    if sorted(label_name_to_id.keys()) == sorted(label_list):
        label_to_id = {i: int(label_name_to_id[label_list[i]]) for i in range(num_labels)}
    else:
        logger.warning(
            "Your model seems to have been trained with labels, but they don't match the dataset: ",
            f"model labels: {sorted(label_name_to_id.keys())}, dataset labels: {sorted(label_list)}."
            "\nIgnoring the model labels as a result.",)

if label_to_id is not None:
    model.config.label2id = label_to_id
    model.config.id2label = {id: label for label, id in config.label2id.items()}

if 128 > tokenizer.model_max_length:
    logger.warning(
        f"The max_seq_length passed ({128}) is larger than the maximum length for the"
        f"model ({tokenizer.model_max_length}). Using max_seq_length={tokenizer.model_max_length}.")
max_seq_length = min(128, tokenizer.model_max_length)

def preprocess_function(examples):
    # Tokenize the texts
    args = (
        (examples[sentence1_key],))
    result = tokenizer(*args, padding=padding, max_length=max_seq_length, truncation=True)

    # Map labels to IDs (not necessary for GLUE tasks)
    if label_to_id is not None and "label" in examples:
        result["label"] = [(label_to_id[l] if l != -1 else -1) for l in examples["label"]]
    return result
raw_datasets = raw_datasets.map(
    preprocess_function,
    batched=True,
    load_from_cache_file=True,
    desc="Running tokenizer on dataset",
)


The OrderedVocab you are attempting to save contains holes for indices [1015, 1016, 1017, 1018, 1053, 1054, 1055, 1056, 1057, 1060, 1061, 1062, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 1076, 1077, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1099, 1101, 1112, 1113, 1556, 1557, 1568], your vocabulary could be corrupted !


Running tokenizer on dataset:   0%|          | 0/35522 [00:00<?, ? examples/s]

The OrderedVocab you are attempting to save contains holes for indices [1015, 1016, 1017, 1018, 1053, 1054, 1055, 1056, 1057, 1060, 1061, 1062, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 1076, 1077, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1099, 1101, 1112, 1113, 1556, 1557, 1568], your vocabulary could be corrupted !


Running tokenizer on dataset:   0%|          | 0/2512 [00:00<?, ? examples/s]

The OrderedVocab you are attempting to save contains holes for indices [1015, 1016, 1017, 1018, 1053, 1054, 1055, 1056, 1057, 1060, 1061, 1062, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 1076, 1077, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1099, 1101, 1112, 1113, 1556, 1557, 1568], your vocabulary could be corrupted !


Running tokenizer on dataset:   0%|          | 0/2512 [00:00<?, ? examples/s]

#### Finalize the training data for training the model

In [40]:
if "train" not in raw_datasets:
    raise ValueError("requires a train dataset")
train_dataset = raw_datasets["train"]
if max_train_samples is not None:
    max_train_samples_n = min(len(train_dataset), max_train_samples)
    train_dataset = train_dataset.select(range(max_train_samples_n))

In [41]:
train_dataset

Dataset({
    features: ['id', 'text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 35522
})

#### Finalize the development/evaluation data for evaluating the model

In [42]:
if "validation" not in raw_datasets:
    raise ValueError("requires a validation dataset")
eval_dataset = raw_datasets["validation"]
if max_eval_samples is not None:
    max_eval_samples_n = min(len(eval_dataset), max_eval_samples)
    eval_dataset = eval_dataset.select(range(max_eval_samples_n))

#### Finalize the test data for predicting the unseen test data using the model

In [43]:
if "test" not in raw_datasets and "test_matched" not in raw_datasets:
    raise ValueError("requires a test dataset")
predict_dataset = raw_datasets["test"]
if max_predict_samples is not None:
    max_predict_samples_n = min(len(predict_dataset), max_predict_samples)
    predict_dataset = predict_dataset.select(range(max_predict_samples_n))

#### Log a few random samples from the training set

In [44]:
for index in random.sample(range(len(train_dataset)), 3):
    logger.info(f"Sample {index} of the training set: {train_dataset[index]}.")

08/18/2025 12:51:17 - INFO - __main__ - Sample 7296 of the training set: {'id': 660, 'text': 'সরকারের দায়িত্বপ্রাপ্ত সংস্থা সকল বাণিজ্যিক ভবন গুলোকে বছরে অন্তত একবার পরিদর্শন করে রিপোর্ট প্রদান করবেন একটা বাণিজ্যিক ভবনে নিরাপত্তার সকল ব্যবস্থা থাকা উচিত কিন্তু এই সব সম্ভবের দেশে কিছুই করা হয় না এই দেশে মানুষের জীবনের কোন দাম নেই তাই কেউ কোনকিছুর তদারকিরও দরকার মনে করে না', 'label': 3, 'input_ids': [101, 2243, 6741, 33983, 59464, 28913, 34687, 67814, 12969, 3079, 100619, 33983, 3012, 3286, 13392, 17618, 2685, 5562, 63129, 3424, 10440, 23054, 5212, 2039, 39178, 2094, 3187, 4804, 2076, 10601, 2506, 2157, 100619, 33983, 3012, 5099, 21962, 19050, 63129, 3187, 3079, 5740, 69734, 2388, 12969, 2187, 2351, 77801, 2045, 2132, 7505, 58136, 2046, 2216, 2178, 7635, 2062, 2047, 33668, 2044, 2045, 2216, 3175, 47678, 2960, 2285, 5803, 2342, 2128, 2174, 2253, 2285, 5803, 6437, 7635, 2046, 18484, 2058, 2659, 2139, 2039, 2044, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

#### Get the metric function `accuracy`

In [45]:
metric = evaluate.load("accuracy")

#### Predictions and label_ids field and has to return a dictionary string to float.

In [46]:
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}


#### Data Collator

In [47]:
data_collator = default_data_collator

#### Initialize our Trainer

In [48]:
train_dataset = train_dataset.remove_columns("id")
eval_dataset = eval_dataset.remove_columns("id")

In [49]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/tmp/ipykernel_1393713/1049306949.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


#### Training our model

In [50]:
train_result = trainer.train()
metrics = train_result.metrics
max_train_samples = (
    max_train_samples if max_train_samples is not None else len(train_dataset)
)
metrics["train_samples"] = min(max_train_samples, len(train_dataset))



[INFO|trainer.py:928] 2025-08-18 12:51:26,921 >> The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
[INFO|trainer.py:2405] 2025-08-18 12:51:26,929 >> ***** Running training *****
[INFO|trainer.py:2406] 2025-08-18 12:51:26,929 >>   Num examples = 35,522
[INFO|trainer.py:2407] 2025-08-18 12:51:26,930 >>   Num Epochs = 15
[INFO|trainer.py:2408] 2025-08-18 12:51:26,930 >>   Instantaneous batch size per device = 16
[INFO|trainer.py:2410] 2025-08-18 12:51:26,930 >>   Training with DataParallel so batch size has been adjusted to: 32
[INFO|trainer.py:2411] 2025-08-18 12:51:26,931 >>   Total train batch size (w. parallel, distributed & accumulation) = 32
[INFO|trainer.py:2412] 2025-08-18 12:51:26,932 >>   Gradient Accumulation steps = 1
[INFO|trainer.py:2413] 2025-08-18 12:51:26,932 >>   Total

Epoch,Training Loss,Validation Loss,Accuracy
1,0.817500,0.762226,0.695462
2,0.710200,0.751082,0.701035
3,0.557200,0.789739,0.700239
4,0.409100,0.900463,0.678742
5,0.271100,1.103938,0.675955
6,0.177800,1.415231,0.670382
7,0.131000,1.608553,0.662022
8,0.088100,1.940637,0.672373
9,0.065200,2.300166,0.664411
10,0.053100,2.540484,0.666003


[INFO|trainer.py:928] 2025-08-18 12:55:06,280 >> The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
[INFO|trainer.py:4258] 2025-08-18 12:55:06,285 >> 
***** Running Evaluation *****
[INFO|trainer.py:4260] 2025-08-18 12:55:06,285 >>   Num examples = 2512
[INFO|trainer.py:4263] 2025-08-18 12:55:06,286 >>   Batch size = 32
[INFO|trainer.py:3942] 2025-08-18 12:55:12,633 >> Saving model checkpoint to ./BanglaBERT_HateSpeech/checkpoint-1111
[INFO|configuration_utils.py:423] 2025-08-18 12:55:12,636 >> Configuration saved in ./BanglaBERT_HateSpeech/checkpoint-1111/config.json
[INFO|modeling_utils.py:3040] 2025-08-18 12:55:13,769 >> Model weights saved in ./BanglaBERT_HateSpeech/checkpoint-1111/model.safetensors
[INFO|tokenization_utils_base.py:2500] 2025-08-18 12:55:13,771 >> tokenizer conf

The OrderedVocab you are attempting to save contains holes for indices [1015, 1016, 1017, 1018, 1053, 1054, 1055, 1056, 1057, 1060, 1061, 1062, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 1076, 1077, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1099, 1101, 1112, 1113, 1556, 1557, 1568], your vocabulary could be corrupted !


[INFO|trainer.py:4034] 2025-08-18 12:55:16,000 >> Deleting older checkpoint [BanglaBERT_HateSpeech/checkpoint-4442] due to args.save_total_limit
/users/t/p/tprama/.local/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
[INFO|trainer.py:928] 2025-08-18 12:58:52,773 >> The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
[INFO|trainer.py:4258] 2025-08-18 12:58:52,777 >> 
***** Running Evaluation *****
[INFO|trainer.py:4260] 2025-08-18 12:58:52,778 >>   Num examples = 2512
[INFO|trainer.py:4263] 2025-08-18 12:58:52,778 >>   Batch size = 32
[INFO|trainer.py:3942] 2025-08-18 12:5

...

[INFO|trainer.py:4034] 2025-08-18 14:03:33,489 >> Deleting older checkpoint [BanglaBERT_HateSpeech/checkpoint-14443] due to args.save_total_limit
/users/t/p/tprama/.local/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
[INFO|trainer.py:3942] 2025-08-18 14:08:06,860 >> Saving model checkpoint to ./BanglaBERT_HateSpeech/checkpoint-16665
[INFO|configuration_utils.py:423] 2025-08-18 14:08:06,863 >> Configuration saved in ./BanglaBERT_HateSpeech/checkpoint-16665/config.json
[INFO|modeling_utils.py:3040] 2025-08-18 14:08:07,964 >> Model weights saved in ./BanglaBERT_HateSpeech/checkpoint-16665/model.safetensors
[INFO|tokenization_utils_base.py:2500] 2025-08-18 14:08:07,966 >> tokenizer config file saved in ./BanglaBERT_HateSpeech/checkpoint-16665/tokenizer_config.json
[INFO|tokeni

The OrderedVocab you are attempting to save contains holes for indices [1015, 1016, 1017, 1018, 1053, 1054, 1055, 1056, 1057, 1060, 1061, 1062, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 1076, 1077, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1099, 1101, 1112, 1113, 1556, 1557, 1568], your vocabulary could be corrupted !


[INFO|trainer.py:4034] 2025-08-18 14:08:10,196 >> Deleting older checkpoint [BanglaBERT_HateSpeech/checkpoint-15554] due to args.save_total_limit
[INFO|trainer.py:928] 2025-08-18 14:08:10,199 >> The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
[INFO|trainer.py:4258] 2025-08-18 14:08:10,202 >> 
***** Running Evaluation *****
[INFO|trainer.py:4260] 2025-08-18 14:08:10,202 >>   Num examples = 2512
[INFO|trainer.py:4263] 2025-08-18 14:08:10,203 >>   Batch size = 32
/users/t/p/tprama/.local/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
[INFO|trainer.py:3942] 2025-08-18 14:

The OrderedVocab you are attempting to save contains holes for indices [1015, 1016, 1017, 1018, 1053, 1054, 1055, 1056, 1057, 1060, 1061, 1062, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 1076, 1077, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1099, 1101, 1112, 1113, 1556, 1557, 1568], your vocabulary could be corrupted !


[INFO|trainer.py:2657] 2025-08-18 14:08:21,673 >> 

Training completed. Do not forget to share your model on huggingface.co/models =)


[INFO|trainer.py:2895] 2025-08-18 14:08:21,674 >> Loading best model from ./BanglaBERT_HateSpeech/checkpoint-2222 (score: 0.7510824799537659).


#### Saving the tokenizer too for easy upload

In [51]:
trainer.save_model()
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

[INFO|trainer.py:3942] 2025-08-18 14:08:33,540 >> Saving model checkpoint to ./BanglaBERT_HateSpeech/
[INFO|configuration_utils.py:423] 2025-08-18 14:08:33,542 >> Configuration saved in ./BanglaBERT_HateSpeech/config.json
[INFO|modeling_utils.py:3040] 2025-08-18 14:08:34,742 >> Model weights saved in ./BanglaBERT_HateSpeech/model.safetensors
[INFO|tokenization_utils_base.py:2500] 2025-08-18 14:08:34,744 >> tokenizer config file saved in ./BanglaBERT_HateSpeech/tokenizer_config.json
[INFO|tokenization_utils_base.py:2509] 2025-08-18 14:08:34,745 >> Special tokens file saved in ./BanglaBERT_HateSpeech/special_tokens_map.json


The OrderedVocab you are attempting to save contains holes for indices [1015, 1016, 1017, 1018, 1053, 1054, 1055, 1056, 1057, 1060, 1061, 1062, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 1076, 1077, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1099, 1101, 1112, 1113, 1556, 1557, 1568], your vocabulary could be corrupted !
***** train metrics *****
  epoch                    =       15.0
  total_flos               = 32642506GF
  train_loss               =     0.2274
  train_runtime            = 1:16:56.06
  train_samples            =      35522
  train_samples_per_second =    115.429
  train_steps_per_second   =       3.61


#### Evaluating our model on validation/development data

In [52]:
logger.info("*** Evaluate ***")

metrics = trainer.evaluate(eval_dataset=eval_dataset)

max_eval_samples = (
    max_eval_samples if max_eval_samples is not None else len(eval_dataset)
)
metrics["eval_samples"] = min(max_eval_samples, len(eval_dataset))

trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

08/18/2025 14:08:41 - INFO - __main__ - *** Evaluate ***


[INFO|trainer.py:928] 2025-08-18 14:08:41,836 >> The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
[INFO|trainer.py:4258] 2025-08-18 14:08:41,839 >> 
***** Running Evaluation *****
[INFO|trainer.py:4260] 2025-08-18 14:08:41,840 >>   Num examples = 2512
[INFO|trainer.py:4263] 2025-08-18 14:08:41,840 >>   Batch size = 32
/users/t/p/tprama/.local/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


***** eval metrics *****
  epoch                   =       15.0
  eval_accuracy           =      0.701
  eval_loss               =     0.7511
  eval_runtime            = 0:00:08.08
  eval_samples            =       2512
  eval_samples_per_second =    310.567
  eval_steps_per_second   =      9.767


### Predecting the test data

In [58]:
id2l = {v: k for k, v in l2id.items()}
logger.info("*** Predict ***")
#predict_dataset = predict_dataset.remove_columns("label")
ids = predict_dataset['id']
predict_dataset = predict_dataset.remove_columns("id")
predictions = trainer.predict(predict_dataset, metric_key_prefix="predict").predictions
predictions = np.argmax(predictions, axis=1)
output_predict_file = os.path.join(training_args.output_dir, f"subtask_1A.tsv")
if trainer.is_world_process_zero():
    with open(output_predict_file, "w") as writer:
        logger.info(f"***** Predict results *****")
        writer.write("id\tlabel\tmodel\n")
        for index, item in enumerate(predictions):
            item = label_list[item]
            item = id2l[item]
            writer.write(f"{ids[index]}\t{item}\t{model_name}\n")
     


08/18/2025 14:15:18 - INFO - __main__ - *** Predict ***


[INFO|trainer.py:928] 2025-08-18 14:15:18,734 >> The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
[INFO|trainer.py:4258] 2025-08-18 14:15:18,738 >> 
***** Running Prediction *****
[INFO|trainer.py:4260] 2025-08-18 14:15:18,739 >>   Num examples = 2512
[INFO|trainer.py:4263] 2025-08-18 14:15:18,739 >>   Batch size = 32


08/18/2025 14:15:25 - INFO - __main__ - ***** Predict results *****


In [59]:
ids[0]

879187

#### Saving the model into card

In [60]:
kwargs = {"finetuned_from": model_name, "tasks": "text-classification"}
trainer.create_model_card(**kwargs)

[INFO|modelcard.py:449] 2025-08-18 14:15:29,611 >> Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}, 'metrics': [{'name': 'Accuracy', 'type': 'accuracy', 'value': 0.7010350227355957}]}


In [62]:
!zip subtask_1A.zip ./BanglaBERT_HateSpeech/task.tsv

	zip warning: name not matched: ./BanglaBERT_HateSpeech/task.tsv

zip error: Nothing to do! (subtask_1A.zip)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
import zipfile
import os
zip subtask_1A.zip ./BanglaBERT_HateSpeech/subtask_1A.tsv